In [ ]:
import findspark
findspark.init()

In [ ]:
import re
import pyspark
from pyspark.sql.types import *

In [ ]:
# Initialize the spark context
sc = pyspark.SparkContext(appName="RDDFromBasics", master="local[4]")

In [ ]:
# Now initialize the spark session
ss = pyspark.sql.SparkSession(sc)

In [ ]:
# Initialize a dataframe reader
dfr = ss.read

In [ ]:
# Define the schema so that it can be passed to the read function
schemaStruct = StructType()
schemaStruct.add("SYMBOL", StringType(), True)
schemaStruct.add("SERIES", StringType(), True)
schemaStruct.add("OPEN", DoubleType(), True)
schemaStruct.add("HIGH", DoubleType(), True)
schemaStruct.add("LOW", DoubleType(), True)
schemaStruct.add("CLOSE", DoubleType(), True)
schemaStruct.add("LAST", DoubleType(), True)
schemaStruct.add("PREVCLOSE", DoubleType(), True)
schemaStruct.add("TOTTRDQTY", LongType(), True)
schemaStruct.add("TOTTRDVAL", DoubleType(), True)
schemaStruct.add("TIMESTAMP", StringType(), True)
schemaStruct.add("ADDNL", StringType(), True)


In [ ]:
# Read the csv file. But first set header to true and ask to infer schema automatically
df = dfr.csv("/home/hduser/spark/nsedata.csv", schema=schemaStruct, header=True)

In [ ]:
# All large number of functions are defines in pyspark.sql.functions.
# import that module and use some of these functions
import pyspark.sql.functions as sqlFns

In [ ]:
df1 = df\
    .where("SYMBOL = 'TCS'")\
    .where("TIMESTAMP like '%2011'")\
    .select(sqlFns.abs(df.OPEN).alias("open"), sqlFns.abs(df.CLOSE).alias("close")).cache()

In [ ]:
df1.printSchema()

In [ ]:
df1.corr("open","close")

In [ ]:
# Now let us run linear regression on this
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression

In [ ]:
df1.rdd.first()

In [ ]:
# Prepare the data for LinearRegression
myTuples = df1.rdd.map(lambda x: (x["close"], Vectors.dense(x["open"])))

In [ ]:
myLRData = ss.createDataFrame(myTuples, ["label", "features"]).cache()

In [ ]:
myLRData.show()

In [ ]:
lr = LinearRegression(standardization=False, fitIntercept=True)

In [ ]:
lrModel = lr.fit(myLRData)
print(type(lrModel))
print(type(lrModel.summary))

In [ ]:
# Now print out all the output parameters generated by linear regression
print("Coefficients: ", lrModel.coefficients)
print("Intercept: ", lrModel.intercept)
print("pValues: ", lrModel.summary.pValues)
print("R-square: ", lrModel.summary.r2)

In [ ]:
lrModel.summary.predictions.show()

In [ ]:
t = lrModel.summary.predictions.collect()

In [ ]:
len(t)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
yold = []
ynew = []
x = []

for i in range(0,len(t)):
    yold.append(t[i]["label"])
    ynew.append(t[i]["prediction"])
    x.append(t[i]["features"])

#print(x,y)

In [ ]:
plt.plot(yold,ynew,"r.")

In [ ]:
plt.plot(x,ynew,"r.")
plt.plot(x,yold,"b.")

In [ ]:
ss.stop()
sc.stop()